# Cleaning methodology for pollutant data

This notebook contains the key steps taken to gather and clean air pollution data. As the data set is quite large and can only be downloaded in small chunks, we are currently investigation different ways of downloading the data in order to increase efficiency. As we continue to explore the data, more substantial filtering and cleaning will take place. 

In [1]:
# import sys
# !{sys.executable} -m pip install django

In [10]:
import pandas as pd
import numpy as np
# from django.urls import path

## Step 1: Identify boundary for monitoring. 

Having manually sifted through the data, we were able to split several monitoring sites into "inner" and "outer" ranges based on their geographic location. The data points closest to our area of focus (Heathrow Airport) have been categorised as "inner" locations and will be vital in our investigation into emissions. The "outer" locations will be used as a way of comparisson to gain a deeper understanding of the scale of the impact arising from air pollution. 

|CCG | Borough| Inner location - monitoring station| Outer location - monitoring site|
|---:|:-----|:-----------|:------------|
|Hillingdon |Hillingdon| Hillingdon South Ruislip, Hillingdon 2 Hillingdon Hospital, Hillingdon Oxford Avenue, Hillingdon Harmondsworth, Hillingdon Harmondsworth Osiris, Hillingdon Hayes, Heathrow LHR2, Heathrow Bath Road, Hillingdon Sipson, Heathrow Green Gates| |
|East Berkshire | Slough |Slough Town Centre Wellington Street,Slough Brands Hill London Road,Slough Windmill Bath Road,Slough Colnbrook,Slough Town Centre A4,Slough Lakeside 1 Osiris,Slough Colnbrook Osiris,Slough Chalvey,Slough Lakeside 2,Slough Lakeside 2 Osiris,Slough - Dennis Way LP11,Slough - Monksfield Way LP20,Slough - The Hawthorns LP2,Slough - Erica Close LP3,Slough - Hatton Avenue LP13,Slough - St Andrews Way LP12,Slough - The Hawthorns LP10,Slough - Francis Way LP13,Slough - The Hawthorns LP1,Slough - Monksfield Way LP19,Slough - Brighton Spur LP3,Slough - Bower Way LP1,Slough - Hatton Avenue LP3,Slough - Cinder Track LP37|
|Hounslow| Hounslow | Hounslow Cranford, Hounslow Chiswick, Hounslow Brentford, Hounslow Heston, Hounslow Hatton Cross, Hounslow Feltham, Hounslow Gunnersbury |
|Ealing | Ealing | Ealing Horn Lane
|Buckinghamshire|South Bucks|Iver Thorney Lane North, Iver North Park Road, Iver Primary School|
|Surrey Heartlands|Richmond|- |Elmbridge|
| - | Spelthorne | Spelthorne Shepperton Squire's Bridge Road, Spelthorne knowle Green, Spelthorne Sunbury Cross, Heathrow Oaks Road|
| - |Waverly and Woking | - | H&F Shepherd’s Bush, Godalming Ockford Road 2|
|South West London| Richmond | London Teddington Bushy Park. |
|Hammersmith & Fulham|London Borough of Hammersmith and Fulham|-| H&F Hammersmith Town Centre, H&F Shepherd’s Bush|
|Watford|Hertfordshire and Bedfordshire|-|Watford Town Hall|
|Oxfordshire|Oxfordshire|-|Oxford High St, Oxford St Ebbes (Cal Club), Oxford Center Roadside, Oxford St Ebbes|
| Berkshire West| Readiing| - | Reading Caversham Road, Reading Oxford Road, Reading London Road, Reading New Town|

## Step 2: Format the Data

In [33]:
boroughs = ['ealing', 'hillingdon', 'richmond', 'slough', 'spelthorn', 'waverly']

In [34]:
li = []

for borough in boroughs:
    df = pd.read_excel('./raw_data/{}.xlsx'.format(borough), header=[0,1], sheet_name=None)
    df = pd.concat(df.values(), axis=0)
    df = pd.DataFrame(df, columns=df.keys())
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [37]:
# frame.to_excel("concatinated_pollution_data.xlsx")

In [ ]:
# df = pd.read_excel('concatinated_pollution_data.xlsx', header=[0,1], sheet_name=None)
# df = pd.concat(df.values(), axis=0)
# df = pd.DataFrame(df, columns=df.keys())
# df.head()

In [10]:
# pd.set_option('display.max_columns', 100)

## Step 3: Unify missing data

In [38]:
df = frame.replace('No data', np.nan)
df = df.replace('No Data', np.nan)
df.head()

Ealing - Acton Town Hall                         \
           Carbon monoxide Nitrogen dioxide Ozone   
0                  0.81494         35.19000   NaN   
1                  0.93136         17.59500   NaN   
2                  0.81494         33.66000   NaN   
3                  1.16420         35.38125   NaN   
4                  1.16420         31.74750   NaN   

                                             \
  PM10 particulate matter (Hourly measured)   
0                                      36.7   
1                                      48.2   
2                                      36.1   
3                                       0.7   
4                                      42.6   

                                                                       \
  PM2.5 particulate matter (Hourly measured) Status Status.1 Status.2   
0                                       13.0    NaN  V ugm-3  V mgm-3   
1                                       15.1    NaN  V ugm-3  V mgm-3   
2                                        8.0    NaN  V ugm-3  V mgm-3   
3                                        8.8    NaN  V ugm-3  V mgm-3   
4                                        2.7    NaN  V ugm-3  V mgm-3   

                           ... Slough Windmill Bath Road           \
        Status.3 Status.4  ...                    Status Status.1   
0  V ugm-3 (VCM)  V ugm-3  ...                       NaN      NaN   
1  V ugm-3 (VCM)  V ugm-3  ...                       NaN      NaN   
2  V ugm-3 (VCM)  V ugm-3  ...                       NaN      NaN   
3  V ugm-3 (VCM)  V ugm-3  ...                       NaN      NaN   
4  V ugm-3 (VCM)  V ugm-3  ...                       NaN      NaN   

  Spelthorne Sunbury Cross                                            \
          Nitrogen dioxide PM10 particulate matter (Hourly measured)   
0                      NaN                                       NaN   
1                      NaN                                       NaN   
2                      NaN                                       NaN   
3                      NaN                                       NaN   
4                      NaN                                       NaN   

                                                                       \
  PM2.5 particulate matter (Hourly measured) Status Status.1 Status.2   
0                                        NaN    NaN      NaN      NaN   
1                                        NaN    NaN      NaN      NaN   
2                                        NaN    NaN      NaN      NaN   
3                                        NaN    NaN      NaN      NaN   
4                                        NaN    NaN      NaN      NaN   

  Unnamed: 0_level_0 Unnamed: 1_level_0  
                Date               Time  
0         2004-01-01           01:00:00  
1         2004-01-01           02:00:00  
2         2004-01-01           03:00:00  
3         2004-01-01           04:00:00  
4         2004-01-01           05:00:00  

[5 rows x 181 columns]

## Step 4: Identify closed monitoring stations

In [39]:
non_null_columns = [col for col in df.columns if df.loc[:, col].notna().any()]
open_monitoring_sites = df[non_null_columns]
open_monitoring_sites

Ealing - Acton Town Hall                         \
                Carbon monoxide Nitrogen dioxide Ozone   
0                       0.81494         35.19000   NaN   
1                       0.93136         17.59500   NaN   
2                       0.81494         33.66000   NaN   
3                       1.16420         35.38125   NaN   
4                       1.16420         31.74750   NaN   
...                         ...              ...   ...   
767440                      NaN              NaN   NaN   
767441                      NaN              NaN   NaN   
767442                      NaN              NaN   NaN   
767443                      NaN              NaN   NaN   
767444                      NaN              NaN   NaN   

                                                  \
       PM10 particulate matter (Hourly measured)   
0                                           36.7   
1                                           48.2   
2                                           36.1   
3                                            0.7   
4                                           42.6   
...                                          ...   
767440                                       NaN   
767441                                       NaN   
767442                                       NaN   
767443                                       NaN   
767444                                       NaN   

                                                                            \
       PM2.5 particulate matter (Hourly measured) Status Status.1 Status.2   
0                                            13.0    NaN  V ugm-3  V mgm-3   
1                                            15.1    NaN  V ugm-3  V mgm-3   
2                                             8.0    NaN  V ugm-3  V mgm-3   
3                                             8.8    NaN  V ugm-3  V mgm-3   
4                                             2.7    NaN  V ugm-3  V mgm-3   
...                                           ...    ...      ...      ...   
767440                                        NaN    NaN      NaN      NaN   
767441                                        NaN    NaN      NaN      NaN   
767442                                        NaN    NaN      NaN      NaN   
767443                                        NaN    NaN      NaN      NaN   
767444                                        NaN    NaN      NaN      NaN   

                                ... Slough Windmill Bath Road           \
             Status.3 Status.4  ...                    Status Status.1   
0       V ugm-3 (VCM)  V ugm-3  ...                       NaN      NaN   
1       V ugm-3 (VCM)  V ugm-3  ...                       NaN      NaN   
2       V ugm-3 (VCM)  V ugm-3  ...                       NaN      NaN   
3       V ugm-3 (VCM)  V ugm-3  ...                       NaN      NaN   
4       V ugm-3 (VCM)  V ugm-3  ...                       NaN      NaN   
...               ...      ...  ...                       ...      ...   
767440            NaN      NaN  ...                       NaN      NaN   
767441            NaN      NaN  ...                       NaN      NaN   
767442            NaN      NaN  ...                       NaN      NaN   
767443            NaN      NaN  ...                       NaN      NaN   
767444            NaN      NaN  ...                       NaN      NaN   

       Spelthorne Sunbury Cross                                            \
               Nitrogen dioxide PM10 particulate matter (Hourly measured)   
0                           NaN                                       NaN   
1                           NaN                                       NaN   
2                           NaN                                       NaN   
3                           NaN                                       NaN   
4                           NaN                                       NaN   
...                         ...                           

In [ ]:
list(open_monitoring_sites.columns.levels[0].unique())

In [ ]:
list(open_monitoring_sites.columns.levels[1].unique())

In [40]:
open_monitoring_sites.to_excel("concatinated_pollution_data.xlsx")

In [1]:
# clean_data = pd.read_excel('concatinated_pollution_data.xlsx', header=[0,1], sheet_name=None)
# clean_dataclean_data = pd.concat(clean_data.values(), axis=0)
# clean_data = pd.DataFrame(clean_data, columns=clean_data.keys())
# clean_data.head()

## Next Steps

- Use API call to gather data from monitoring stations
- Investigate other data sources for mass downloads and ease
- Aggregate data on a daily basis
- Investigate similarities/differences between outer and inner locations, perform EDA on each
- Aggregate data annually and merge with Health and Flight data
